In [1]:
from functools import reduce

from cycler import cycler
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from numbers import Number
import numpy as np
import pandas as pd
#from scipy import stats as sstats
import wmfdata as wmf
from wmfdata.utils import pct_str, sig_figs
from wmfdata.charting import pct_fmt, M_fmt, comma_fmt

import gspread
from google.oauth2.service_account import Credentials

You are using wmfdata v1.3.3, but v2.0.0 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


In [2]:
FILE = "metrics/metrics.tsv"
metrics = pd.read_csv(FILE, sep="\t", parse_dates=["month"]).set_index("month")

In [3]:
FILE = "metrics/diversity_metrics.tsv"
diversity_metrics = pd.read_csv(FILE, sep="\t", parse_dates=["month"]).set_index("month")

# Report

In [4]:
metrics_all = pd.concat([metrics, diversity_metrics], axis=1)
metrics_all  = metrics_all.loc[:,~metrics_all.columns.duplicated()]

In [5]:
report_order = [
    'interactions', 
        'total_pageview',
        'automated_pageviews',
        'desktop', 
        'mobileweb',
        'previews_seen',
    'unique_devices',
    # Diversity
    'gn_interactions',
    'gs_interactions'
    #'Mobile-heavy wikis'
    #'mh_interactions'
]

In [6]:
def fmt_num(x):
    if isinstance(x, Number) and not pd.isnull(x):
        x = sig_figs(x, 3)
        
        if x < 5:
            return pct_str(x)
        else:
            return "{:,.0f}".format(x)
    else:
        return x

def calc_rpt(ser):   
    cur = ser[-1]
    
    try:
        yr_prev = ser[-13]
        yoy_change = (cur / yr_prev) - 1
    except IndexError:
        yoy_change = None
    
        
    
    res = [cur, yoy_change]
    return pd.Series(
        [fmt_num(n) for n in res],
        index=["value", "yoy_change"]
    )

In [7]:
(
    metrics_all
    .apply(calc_rpt)
    .transpose()
    .reindex(report_order)
    .fillna("—")
)

,value,yoy_change
interactions,"21,000,000,000",18.4%
total_pageview,"19,300,000,000",21.5%
automated_pageviews,"3,130,000,000",255.0%
desktop,"6,640,000,000",3.4%
mobileweb,"12,400,000,000",34.5%
previews_seen,"1,730,000,000",-8.2%
unique_devices,"1,600,000,000",-7.4%
gn_interactions,"15,300,000,000",13.2%
gs_interactions,"5,740,000,000",35.1%


# Update Google Sheet

In [8]:
import gspread
from google.oauth2.service_account import Credentials

In [9]:
FILE = "metrics/metrics.tsv"
all_metrics = pd.read_csv(FILE, sep="\t")

DIV_FILE = "metrics/diversity_metrics.tsv"
diversity_metrics = pd.read_csv(DIV_FILE, sep="\t")


In [10]:
all_metrics.iloc[-1,1:] = all_metrics.iloc[-1,1:].astype(float)
diversity_metrics.iloc[-1,1:] = diversity_metrics.iloc[-1,1:].astype(float)

In [12]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('key-metrics-343717-07427b76df93.json', scopes=scope)
gc = gspread.authorize(credentials)

In [13]:
@property
def sheet1(self):
    """Shortcut property for getting the first worksheet."""
    return self.get_worksheet(0)

In [14]:
wks1 = gc.open("readers").sheet1
new_metrics = all_metrics.iloc[-1,[0,1,2,3,4,5,6,7]].tolist()
wks1.append_row(new_metrics)

{'spreadsheetId': '1mK-R8qWzKjSeHMBBek9sJsbecdic9s3r28OIW7QkqrE',
 'tableRange': 'metrics!A1:H77',
 'updates': {'spreadsheetId': '1mK-R8qWzKjSeHMBBek9sJsbecdic9s3r28OIW7QkqrE',
  'updatedRange': 'metrics!A78:H78',
  'updatedRows': 1,
  'updatedColumns': 8,
  'updatedCells': 8}}

In [15]:
# Depricated as we are no longer calculating mobile-heavy metrics
#wks2 = gc.open("readers").get_worksheet(1)
#mh_metrics = all_metrics.iloc[-1,[0,7,8,9]].tolist()
#wks2.append_row(mh_metrics)

In [16]:
wks3 = gc.open("readers").get_worksheet(1)
div_metrics = diversity_metrics.iloc[-1].tolist()
wks3.append_row(div_metrics)

{'spreadsheetId': '1mK-R8qWzKjSeHMBBek9sJsbecdic9s3r28OIW7QkqrE',
 'tableRange': 'diversity_global_market!A1:G77',
 'updates': {'spreadsheetId': '1mK-R8qWzKjSeHMBBek9sJsbecdic9s3r28OIW7QkqrE',
  'updatedRange': 'diversity_global_market!A78:G78',
  'updatedRows': 1,
  'updatedColumns': 7,
  'updatedCells': 7}}